In [1]:
import folium
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import decimal
from shapely import wkt
from pyproj import Proj, transform
from haversine import haversine

In [10]:
df_ins = pd.read_csv('./data/202011_자연환경조사_곤충_점.csv')
df_rep = pd.read_csv('./data/202011_자연환경조사_양서파충류_점.csv')
df_spi = pd.read_csv('./data/202011_자연환경조사_저서무척추동물_점.csv')
df_bird = pd.read_csv('./data/202011_자연환경조사_조류_점.csv')
df_mam = pd.read_csv('./data/202011_자연환경조사_포유류_점.csv')

In [44]:
df_bird

,spce_id,examin_realm_se_code_nm,spcs_lcnm,spcs_scncenm,examin_year,examin_begin_de,examin_end_de,geom,lati,long
0,8000000002190754,조류,유리딱새,"Luscinia cyanura (Pallas, 1773)",2017,2017-11-23,2017-11-23,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
1,8000000002160958,조류,유리딱새,"Luscinia cyanura (Pallas, 1773)",2016,2016-09-20,2016-09-20,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
2,8000000002191375,조류,유리딱새,"Luscinia cyanura (Pallas, 1773)",2017,2017-06-05,2017-06-05,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
3,8000000002176360,조류,유리딱새,"Luscinia cyanura (Pallas, 1773)",2017,2017-04-22,2017-04-22,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
4,8000000002206050,조류,유리딱새,"Luscinia cyanura (Pallas, 1773)",2017,2017-04-08,2017-04-08,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
...,...,...,...,...,...,...,...,...,...,...
264036,8000000002246386,조류,쇠유리새,"Luscinia cyane (Pallas, 1776)",2018,2018-05-14,2018-05-14,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
264037,8000000002193688,조류,쇠유리새,"Luscinia cyane (Pallas, 1776)",2017,2017-05-11,2017-05-11,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
264038,8000000002335473,조류,쇠유리새,"Luscinia cyane (Pallas, 1776)",2017,2017-07-06,2017-07-06,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2
264039,8000000002228436,조류,쇠유리새,"Luscinia cyane (Pallas, 1776)",2018,2018-05-15,2018-05-15,POINT (1844674401947755.2 1844674393761355.2),1844674393761355.2,1844674401947755.2


In [43]:
inProj = Proj(init='epsg:2097')
outProj = Proj(init='epsg:4326')

kr_long, kr_lati = transform(inProj,outProj, list(df_bird['lati']), list(df_bird['long']))
kr_long[0], kr_lati[0]

(inf, inf)

In [11]:
df_ins['lati'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_ins['long'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_rep['lati'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_rep['long'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_spi['lati'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_spi['long'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_bird['lati'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_bird['long'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_mam['lati'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_mam['long'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

In [4]:
df_all = pd.concat([df_ins, df_rep, df_spi, df_mam]).copy()
df_all = df_all.reset_index(drop=True)

In [5]:
inProj = Proj(init='epsg:5186')
outProj = Proj(init='epsg:4326')

kr_long, kr_lati = transform(inProj,outProj, list(df_all['lati']), list(df_all['long']))

df_all['kr_lati'] = kr_lati
df_all['kr_long'] = kr_long

In [6]:
user = (35.233512, 129.0722714)

In [9]:
df_all.iloc[288510]

spce_id                                                 9000000000567234
examin_realm_se_code_nm                                              포유류
spcs_lcnm                                                            고라니
spcs_scncenm                            Hydropotes inermis Swinhoe, 1870
examin_year                                                         2016
examin_begin_de                                               2016-06-05
examin_end_de                                                 2016-06-05
geom                       POINT (1844674401947755.2 1844674393761355.2)
lati                                                  1844674393761355.2
long                                                  1844674401947755.2
kr_lati                                                              inf
kr_long                                                              inf
Name: 288510, dtype: object

In [8]:
want_list = []
for i in range(len(df_all)):
    try:
        if haversine(user, (df_all['kr_lati'][i], df_all['kr_long'][i]), unit='km') <= 10 :
            want_list.append(i)
            
    except : print(i)

275754
276615
277768
277948
278373
279327
279715
280864
281258
281948
281990
282681
282716
283238
283333
283360
283638
283677
283886
284016
284020
284099
284382
284407
284430
284442
284478
284490
284503
284675
284793
284799
284862
284940
285099
285220
285227
285272
285320
285426
285451
285515
285585
285788
285962
286043
286059
286116
286133
286181
286207
286323
286365
286401
286540
286594
286668
286763
286785
286950
286976
287078
287098
287123
287213
287319
287404
287556
287605
287623
287637
287639
287688
287738
287856
287968
287983
288072
288137
288207
288267
288334
288375
288510
288845
288934
288935
288936
288937
288938
288939
288940
288941
288942
288943
288944
288945
288946
288947
288948
288949
288950
288951
288952
288953
288954
288955
288956
288957
288958
288959
288960
288961
288962
288963
288964
288965
288966
288967
288968
288969
288970
288971
288972
288973
288974
288975
288976
288977
288978
288979
288980
288981
288982
288983
288984
288985
288986
288987
288988
288989
288990
288991

In [164]:
want_list

'좀남색잎벌레'

In [161]:
want_list[0]

136

In [171]:
m = folium.Map(location=[user[0], user[1]], zoom_start = 12)

tooltip = '나를 눌러줘!'

for i in want_list:
    folium.CircleMarker(location=[df_ins['kr_lati'][i], df_ins['kr_long'][i]], 
                        popup = '<b>'+df_ins.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                        radius=3, color='green').add_to(m)

m

In [77]:
m.save('map.html')